In [1]:
# 1. Importaciones necesarias
import cv2
import math
import easyocr  # Para el Reconocimiento Óptico de Caracteres (OCR)
from ultralytics import YOLO
import pandas as pd
from datetime import datetime

c:\Users\Mohamed O. Haroun\miniconda3\envs\VC_P4\lib\site-packages\torchvision\io\image.py:14: UserWarning: Failed to load image Python extension: 'Could not find module 'C:\Users\Mohamed O. Haroun\miniconda3\envs\VC_P4\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [ ]:
 # 2. Configuración del sistema LPR


# Cargar tu modelo entrenado para detectar matrículas (best.pt)

# Asegúrate de que la ruta sea correcta

MODELO_MATRICULA = 'yolo11n_best.pt'

model = YOLO(MODELO_MATRICULA)


# Inicializar el lector EasyOCR para español e inglés (o solo el que necesites)

# Nota: La primera vez que se ejecuta, descarga los modelos.

reader = easyocr.Reader(['es', 'en'])


# Nombre del archivo de video a procesar

VIDEO_FILENAME = "plates_test.mp4"


# Crear una lista para almacenar los datos del CSV

data_for_csv = []

current_frame = 0


# 3. Inicializar la captura de video

vid = cv2.VideoCapture(VIDEO_FILENAME)

# Para seguimiento y registro de tiempo:

fps = vid.get(cv2.CAP_PROP_FPS)


# Colores para dibujar (BGR)

COLOR_RECTANGULO = (0, 255, 0) # Verde

COLOR_TEXTO = (255, 255, 255) # Blanco


# 4. Bucle principal de procesamiento de video

while vid.isOpened():

    ret, img = vid.read()

    current_frame += 1


    if not ret:

        break # Salir si no hay más fotogramas


    # **Paso A: Detección y Seguimiento (YOLO)**

    # Usamos .track() para mantener un ID constante para cada vehículo

    results = model.track(img, persist=True, tracker="bytetrack.yaml", verbose=False)

   

    # Calcular el tiempo del fotograma en el video

    timestamp_ms = (current_frame / fps) * 1000

    timestamp_str = str(datetime.fromtimestamp(timestamp_ms / 1000.0).strftime('%H:%M:%S.%f'))[:-3]


    # Procesar las detecciones

    if results and results[0].boxes:

        boxes = results[0].boxes

       

        for box in boxes:

            # Obtener coordenadas, confianza y ID

            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())

            confidence = round(box.conf[0].item(), 2)

            track_id = int(box.id[0].item()) if box.id is not None else -1

                       

            padding = 5

            x1_crop = max(0, x1 - padding)

            y1_crop = max(0, y1 - padding)

            x2_crop = min(img.shape[1], x2 + padding)

            y2_crop = min(img.shape[0], y2 + padding)

           

            placa_recortada = img[y1_crop:y2_crop, x1_crop:x2_crop]

           

            matricula_texto = "Matricula NO LEIDA"

            try:

                # Usar EasyOCR para leer la imagen recortada

                # Usamos detalles finos y un umbral de confianza alto para evitar ruido

                ocr_results = reader.readtext(placa_recortada, detail=0, allowlist='0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ', min_size=5)

               

                if ocr_results:

                    # Usamos el resultado con más caracteres (asumiendo que es la matrícula)

                    matricula_texto = max(ocr_results, key=len).replace(" ", "")

               

            except Exception as e:

                # print(f"Error en OCR: {e}")

                pass # Continuar si el OCR falla en un fotograma


            # **Paso C: Visualización y Registro**

           

            # 1. Dibujar el rectángulo de detección

            cv2.rectangle(img, (x1, y1), (x2, y2), COLOR_RECTANGULO, 2)

           

            # 2. Preparar el texto para mostrar (ID de seguimiento y Matrícula)

            display_text = f"ID: {track_id} | Matrícula: {matricula_texto} ({confidence:.2f})"

           

            # 3. Dibujar el texto encima del rectángulo

            cv2.putText(img, display_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, COLOR_RECTANGULO, 2)

           

            # 4. Almacenar datos para el CSV

            data_for_csv.append({

                'Frame': current_frame,

                'Tiempo (HH:MM:SS.ms)': timestamp_str,

                'Tracking ID': track_id,

                'Matrícula Leída': matricula_texto,

                'Confianza YOLO': confidence,

                'Coordenadas (x1, y1, x2, y2)': f"({x1}, {y1}, {x2}, {y2})"

            })

           

    # Mostrar el fotograma procesado

    cv2.imshow('LPR en Video', img)

   

    # Salir con ESC (27)

    if cv2.waitKey(int(1000/fps)) & 0xFF == 27:

        break


# 5. Liberar recursos y generar el CSV

vid.release()

cv2.destroyAllWindows()


# Crear y guardar el archivo CSV

df = pd.DataFrame(data_for_csv)

df.to_csv('resultados_lpr.csv', index=False)


print(f"\n✅ Procesamiento de video terminado.")

print(f"✅ Resultados guardados en resultados_lpr.csv") 


✅ Procesamiento de video terminado.
✅ Resultados guardados en resultados_lpr.csv
